### Training a ground-up MLP in native Julia (yes, on MNIST).

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png" width='50%' height='50%' ></img>

**Part 1** will focus on building from the ground-up a neural network capable of a simple forward pass. We’ll train it with the backpropagation algorithm in **Part 2**, and evaluate it in **Part 3**.

#### Part 1 — A forward pass

$$y = g( W_3 f ( W_2 f( W_1 X+ b_1 ) + b_2 ) + b_3)$$

In which $W_n$ are our weight matrices, and $b_n$ are our bias vectors. The activation functions $f, g$ we use in this model are a **ReLU** and a **Softmax** respectively. $y$ (our prediction) approximates $Y$ (the true value) as a function of observations (pixel values) $X$.

#### Building blocks

Julia is not really meant to be used as an object-oriented language like Python. While you can define a _struct_, this is a `"composite data type”` rather than a **Python** _object_, and does not have _methods_ defined in its definition. There’s no sense of calling `neural_net.forward()` in which we can access `neural_net.attribute_1`, for example.

Instead, in **Julia** we rely on `“dynamic dispatch”`, in which _functions_ are defined uniquely for varied input data types. So `forward(x:Array{Float64,2})` , can call completely different code to `forward(x:Int)`. The concept is similar to `“overloading”` functions in eg. **Java**, but as [this](https://stackoverflow.com/questions/50583861/overloading-vs-overriding-in-julia) interesting **StackOverflow** post points out, overloading corresponds to deciding the function to be called at compile time, while in **Julia** we decide at run time.

So, our key MLP building block is the dense layer:
```julia
mutable struct Dense
    # Fully connected layer. By default has weights, biases, and an activation function.
    weight
    bias
    act_fn
    Dense(dim_in, dim_out, act_fn)=new(kaiming(Float64, dim_out, dim_in), 
          zeros(Float64, dim_out, 1), act_fn)
end;
```

In [3]:
include("components_dense.jl");

We implement this as a _struct_ , with a _weight_, _bias_ , and an _anact_fn_. Note that importantly it’s _mutable_, its attributes (the **MLP parameters**) are allowed to change. For quality of life we don’t want to instantiate this layer using the syntax: `layer_1 = Dense(W_1, b_1, ReLU())`, as this would require defining the matrix and vector elsewhere, and that’s messy. Instead, we overload `Dense()` to take dimensionality integers as input, and generate in-place a new **Kaiming init’d matrix** for the weights, and zeros for the bias.

In [4]:
include("components_kaiming.jl");

For an instance _layer_ of _Dense_, we can now call _layer.weight_ , _layer.bias_ , _layer.act_fn_ as we require.

#### Non-linearity

For the activation functions we also utilise a _struct_, with dynamically dispatched _forward()_ and _gradient()_ methods. Every binary operator in **Julia** has a vectorised version denoted by the _._ prefix. This is similar to behaviour in MATLAB, and one of my personal favourite features of that language. These are very helpful for implementing _pointwise activation functions_.

```julia
struct ReLU
end;
forward(z::Array{Float64, 2}, act_fn::ReLU)::Array{Float64,2} = z.*(z.>0)
gradient(z::Array{Float64, 2}, act_fn::ReLU)::Array{Float64,2} = Array{Float64, 2}(z.>0)

struct Softmax
end;
forward(z::Array{Float64, 2}, act_fn::Softmax)::Array{Float64,2} = softmax(z)
gradient(z::Array{Float64, 2}, act_fn::Softmax)::Array{Float64,2} = (softmax(z)) .* (1 .- softmax(z))

function softmax(z::Array{Float64,2})::Array{Float64,2}
    #converts real numbers to probabilities
    c=maximum(z)
    p = z .- log.( sum( exp.(z .- c) ) ) .-c
    p = exp.(z)
    return p
end
```

In [8]:
include("activations.jl");

The ReLU function’s forward method simply returns the input value if it’s more than zero, else zero. The ${\color{Yellow}gradient}$ is one if the input is more than zero, else zero.

$$\text{softmax}(z)_i = \frac{e^{z_i}}{\sum_{j=1}^m e^{z_j}}$$
$$\text{The softmax function, element-wise}$$

The softmax function acting on a vector, _softmax(z)_, returns a normalised pointwise exponential of the vector’s elements, such that the output elements sum to one. Its derivative with respect to its input _z_, takes the form _grad_softmax(z) = softmax(z) * (1 — softmax(z))_ , where 1 is the appropriate vector of ones, and * is the Hadamard product ( _.*_ in Julia). In this implementation, we utilise the so-called ‘log-sum-exp’ trick for numerical stability when calculating the softmax. There’s a great explanation of this technique [here](https://blog.feedly.com/tricks-of-the-trade-logsumexp/).

The softmax function ensures that an output vector’s elements are all > 0, real valued, and sum to one. These properties are synonymous with a valid probability distribution over the vector’s elements, which we choose to use as a model for the probabilty of an input vector **x**’s assignment to one of the classes represented by the output vector **y**’s elements. Under this model, the inputs to the softmax function are the log-odds associated with each class. This is essentially all we need to build the functional form of our MLP.

#### Putting it together

When training the neural network we’ll need to keep track of intermediate activations as well as the values of the parameters themselves, and here we simply utilise a _dict_ (to allow O(1) access to any element) to do so:

```julia
# The net is simply a dictionary containing parameters,
# activations post- (A) and pre- (Z) activation function.
net=Dict("Layers"=>[], "A"=>[], "Z"=>[])

dims=[[28^2, 32] [32, 32] [32, 10]]
layers=[]
for i in 1:size(dims,2)-1
    append!(layers, [Dense(dims[1,i], dims[2,i], ReLU())])
end

head=[Dense(dims[1, size(dims,2)], dims[2, size(dims,2)], Softmax())]
append!(layers, head);
net["Layers"]=layers;
```

In [11]:
include("neural_net.jl");

So here we define a net dictionary, with fields “Layers”, “A”, and “Z”. “Z” and “A” are required for the backpropagation algorithm, as you’ll see later.

- “Layers” is a list of the individual Dense layers.
- “A” tracks the values of the layers’ activations through a forward pass after applying the activation function.
- “Z” tracks the activations prior to applying the activation function.

Structurally, the final thing we need to do is define a _forward()_ method for our MLP, again utilising native **Julia** dynamic dispatch. We simply loop through the stored layers in our _dict_, multiplying the input by the weight matrix, adding the bias, and applying our pointwise activation function. We keep track of the activations both before and after applying the activation function.

```julia
function forward(x::Array{Float64,2}, net::Dict)::Array{Float64,2}
    # Feed the input through a network.
    # Keep track of the activations both pre- and post- activation function.
    # Store these values in the network's dictionary.
    A=[x]
    Z=[]
    for n in 1:length(net["Layers"])
        z = net["Layers"][n].weight*x + net["Layers"][n].bias
        x = forward(z, net["Layers"][n].act_fn)
        append!(Z, [z])
        append!(A, [x])
    end
    net["A"]=A
    net["Z"]=Z
    return x
end; 
```

In [12]:
include("forward.jl");

So we’re in a good place. We can call _y = forward(x, net)_ to generate a set of predictions for the class of _x_, though this is obviously random at the moment as we have yet to train the network. Time for backpropagation.

## Part 2 — Training

We have built the functional form of our MLP: _y = forward(x, net)_ ouputs a 10 x 1 vector _y_ from a 784 x 1 vector _x_. We’ll go into the backpropagation algorithm for updating the net’s parameters shortly, but first we need to talk data.

#### Loading and loss

After installing, we can import with _using MLDatasets_. We can then _calltrain_x, train_y = MNIST.traindata(Float64)_. The _x_ data is good to go from here, but we need to use “one-hot encoding” for the _y_ data, as _train_y_ by default outputs an integer between 0 and 9 as the class label. We do this using the function _zi_one_hot_encode(train_y)_ :

```julia
function zi_one_hot_encode(data)
  
    # assumes zero-indexed categories
    one_hot=zeros(Float64, maximum(data)+1, size(data, 1));
  
    for i in 1:size(data, 1)
        label=data[i]+1
        one_hot[label, i]=1
    end;

    return one_hot

end;
```

In [13]:
include("zi_one_hot.jl");

As discussed previously, our _Softmax_ layer outputs a vector representing a probability distribution over classes, ie. its elements are real valued, > 0, and sum to one. We would like the output of our MLP, a vector y representing a probability distribution over classes, to be as close possible to the one-hot encoding of the training data. We can then minimise the cross-entropy between these probability distributions to train our neural network.

$$\mathcal{L} = -\frac{1}{m} \sum_{i=1}^m Y_i \; log \; y_i$$
$$\text{Cross-entropy loss}$$

For a deeper dive into the cross-entropy loss, and its (close) relation to the Kullback-Leibler divergence (the _relative_ entropy), see [here](https://machinelearningmastery.com/cross-entropy-for-machine-learning/). It’s the maximum likelihood function for a Bernoulli distribution over classes (corresponding to our one-hot encoding), so is the correct loss function for single-label classification. Here’s a Julia implementation, along with its derivative which we’ll need for backpropagation:

```julia
# xe_losses assume inputs as probabilities (real valued and 
# normalised, ie. after a Softmax).
xe_loss(y::Array{Float64,2}, Y::Array{Float64,2}) = - sum(Y.*log.(y))
xe_loss_derivative(y::Array{Float64,2}, Y::Array{Float64,2}) = y - Y
```

In [15]:
include("cross_entropy.jl");

So we’re finally ready to tackle the big beast — backpropagation. I found [this](https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795) blog post by Piotr Skalski tremendously helpful when tackling this, and a lot of my implementation of this algorithm is inspired by his work. I’ve used the same syntax, so check out his explanation if mine doesn’t do it for you!

# References
- [ ] [Training a ground-up MLP in native Julia (yes, on MNIST).](https://tmcauliffe.medium.com/training-a-ground-up-mlp-in-native-julia-yes-on-mnist-c2f84aaca2f5)
- [ ] [MNIST database](https://en.wikipedia.org/wiki/MNIST_database)